<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=270651362" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv
/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kagg

In [5]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-v4/user_course_v4.csv'
df_user_video_final_path = '/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv'

In [6]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg, regexp_replace
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/25 03:15:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [8]:
df_user_video=spark.read.json(BASE_DIR + "/user-video.json")
# df_user_video.show()

In [9]:
# df_user_video.printSchema()

In [10]:
from pyspark.sql.functions import col, explode, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Giải nén `seq` để mỗi dòng có một `video_id`
df_flatten = df_user_video.withColumn("seq", explode(col("seq")))

# Giải nén tiếp `segment` để mỗi dòng là một đoạn xem video
df_flatten = df_flatten.withColumn("segment", explode(col("seq.segment")))

# Chọn các cột cần thiết
df_flatten = df_flatten.select(
    col("user_id"),
    col("seq.video_id").alias("video_id"),
    col("segment.start_point"),
    col("segment.end_point"),
    col("segment.speed"),
    col("segment.local_start_time")
)
# df_flatten.show(truncate=True)

In [11]:
from pyspark.sql.functions import from_unixtime

df_flatten = df_flatten.withColumn("local_start_time", from_unixtime("local_start_time", "yyyy-MM-dd HH:mm:ss"))
# df_flatten.show(truncate=False)

In [12]:
from pyspark.sql.functions import regexp_replace, col
df_cleaned = df_flatten.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

# df_cleaned.show(5, truncate=False)


In [13]:
df_cleaned.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- video_id: long (nullable = true)
 |-- start_point: double (nullable = true)
 |-- end_point: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- local_start_time: string (nullable = true)



In [14]:
df_cleaned = df_cleaned.filter(col("start_point") <= col("end_point"))


In [15]:
df_course = spark.read.json(BASE_DIR + '/course.json')
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)
df_video_course_map_clean = df_video_course_map.withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
).withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)

# df_video_course_map_clean.show()

In [16]:
df_video = spark.read.json(BASE_DIR + "/video.json")
from pyspark.sql.functions import array_max
df_video = (
    df_video
    .select(
        "ccid", 
        array_max("end").alias("video_duration")
    )
)
# df_video.show()

In [17]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
# Parse mapping file
df_video_id_ccid_v2 = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
video_info = df_video_id_ccid_v2.join(df_video, on=['ccid'])
video_info_cleaned = video_info.withColumn(
    "video_id", regexp_replace("video_id", "^V_", "").cast("long")
)
# video_info_cleaned.show(truncate=False)

In [18]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course_clean = df_user_course.withColumn(
    "user_id", regexp_replace("user_id", "^U_", "").cast("long")
).withColumn(
    "course_id", regexp_replace("course_id", "^C_", "").cast("long")
)

# df_user_course_clean.show()

In [19]:
count = df_user_course_clean.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_course_clean.columns
])
# count.show(truncate=False)

In [20]:
df_uv_uc =  df_user_course_clean.join(
    df_cleaned,
    on='user_id',
    how='left'
)
# df_uv_uc.show()

In [21]:
df_uv_uc.count()

37695451

In [22]:
count = df_uv_uc.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_uv_uc.columns
])
# count.show(truncate=False)

#  ----- 1. Gộp video_info_cleaned -----

In [23]:
# Sử dụng 'left' join để giữ tất cả các hàng từ df_uv_uc,
# kể cả khi video_id là NULL hoặc không tìm thấy trong video_info_cleaned.
df_step1 = df_uv_uc.join(
    video_info_cleaned,
    on='video_id',
    how='left'
)

# ----- 2. Thêm cột since_day_enroll -----

In [24]:
from pyspark.sql.functions import col, datediff, when, abs
df_step2 = df_step1.withColumn(
    "since_day_enroll",
    abs(datediff(col("local_start_time"), col("enroll_time")))
)

# ----- 3. Thêm các cột phase 1, 2, 3, 4 -----

In [25]:
# Định nghĩa các khoảng ngày cho mỗi phase (mỗi phase 14 ngày)
# 4 phases * 14 ngày = 56 ngày. Chúng ta sẽ coi tổng phạm vi hợp lệ là từ 0 đến 55.
phase_1_cond = col("since_day_enroll").between(0, 13)
phase_2_cond = col("since_day_enroll").between(14, 27)
phase_3_cond = col("since_day_enroll").between(28, 41)
phase_4_cond = col("since_day_enroll").between(42, 55)

# Định nghĩa điều kiện "hoặc lớn hơn --> NULL" hoặc "không có giá trị --> NULL"
# Bất kỳ giá trị since_day_enroll nào là NULL, < 0, hoặc > 55 sẽ bị coi là không hợp lệ.
is_valid_range = col("since_day_enroll").isNotNull() & col("since_day_enroll").between(0, 55)

# Tạo các cột phase
# Nếu không nằm trong phạm vi hợp lệ (is_valid_range = false) -> NULL
# Nếu nằm trong phạm vi, kiểm tra điều kiện của phase đó:
#   - Nếu đúng (ví dụ: ngày 10 cho phase 1) -> 1
#   - Nếu sai (ví dụ: ngày 20 cho phase 1) -> 0
df_final = df_step2 \
    .withColumn("phase_1",
        when(~is_valid_range, None)
        .when(phase_1_cond, 1)
        .otherwise(0)
    ) \
    .withColumn("phase_2",
        when(~is_valid_range, None)
        .when(phase_2_cond, 1)
        .otherwise(0)
    ) \
    .withColumn("phase_3",
        when(~is_valid_range, None)
        .when(phase_3_cond, 1)
        .otherwise(0)
    ) \
    .withColumn("phase_4",
        when(~is_valid_range, None)
        .when(phase_4_cond, 1)
        .otherwise(0)
    )

# Hiển thị kết quả
# df_final.show()

In [26]:
# df_final.count()

In [27]:
phase_counts = df_final.agg(
    sum("phase_1").alias("count_phase_1"), 
    sum("phase_2").alias("count_phase_2"),
    sum("phase_3").alias("count_phase_3"),
    sum("phase_4").alias("count_phase_4"),
    
)

In [28]:
# phase_counts.show()

In [29]:
from pyspark.sql.window import Window 
from pyspark.sql.functions import col, lit, when, sum, avg, count, lag, lead, round 
import pyspark.sql.functions as f

# 1. Tạo các đặc trưng cơ bản

In [30]:
df_features = (
    df_final
    .withColumn("watch_duration", col("end_point") - col("start_point"))
    .withColumn("watch_duration_real", col("watch_duration") / col("speed"))
)

print("Added watch_duration and watch_duration_real")

Added watch_duration and watch_duration_real


# 2. Định nghĩa các cửa sổ (Windows)

In [38]:
# 2.1
window_user_course = Window.partitionBy("user_id", "course_id")
# 2.2
window_video_temporal = (
    Window.partitionBy("user_id", "course_id", "video_id")
    .orderBy("local_start_time")
)
print("defined window_user_course and window_video_temporal")


defined window_user_course and window_video_temporal


# 3. Tạo đặc trưng thống kê tổng quan

In [39]:
# 3.1. Tính toán các thống kê tổng quan
df_features = (
    df_features
    .withColumn("total_watch_duration_real_user", F.round(F.sum("watch_duration_real").over(window_user_course), 2))
    .withColumn("total_sessions_user", F.count("*").over(window_user_course)) # Dòng này đã đúng
    .withColumn("avg_speed_user", F.round(F.avg("speed").over(window_user_course), 2))
    .withColumn("avg_watch_duration_real_user", F.round(F.avg("watch_duration_real").over(window_user_course), 2))
)
# df_features.show(20, truncate=False)

# 4. Tạo đặc trưng về hành vi (Pause/Seek))

In [40]:
# 4. Tạo đặc trưng về hành vi (Pause/Seek))
# 4.1. Lấy end_point của segment TRƯỚC ĐÓ
df_features = df_features.withColumn("prev_end_point", F.lag("end_point").over(window_video_temporal))

# 4.2. Tính toán khoảng chênh lệch (gap)
df_features = df_features.withColumn("gap", 
    F.when(col("prev_end_point").isNull(), 0)
     .otherwise(col("start_point") - col("prev_end_point"))
)

# 4.3. Phân loại hành vi cho TỪNG SEGMENT
df_features = (
    df_features
    .withColumn("is_pause", F.when((col("gap") >= 0) & (col("gap") <= 2), 1).otherwise(0)) 
    .withColumn("is_seek_forward", F.when(col("gap") > 2, 1).otherwise(0)) 
    .withColumn("is_seek_backward", F.when(col("gap") < 0, 1).otherwise(0))
)
# 4.4. Đếm TỔNG SỐ lần hành vi cho mỗi user/course
df_features = (
    df_features
    .withColumn("total_pauses_user", F.sum("is_pause").over(window_user_course)) 
    .withColumn("total_seek_forward_user", F.sum("is_seek_forward").over(window_user_course)) 
    .withColumn("total_seek_backward_user", F.sum("is_seek_backward").over(window_user_course))
)
print("Đã thêm các đặc trưng hành vi (pause/seek):")

Đã thêm các đặc trưng hành vi (pause/seek):


# 5. Tạo đặc trưng về tốc độ xem 

In [42]:
# 5.1. Tạo cờ cho từng loại tốc độ
df_features = (
    df_features
    .withColumn("is_speed_1", F.when(col("speed") == 1.0, 1).otherwise(0)) 
    .withColumn("is_speed_1_25", F.when(col("speed") == 1.25, 1).otherwise(0)) 
    .withColumn("is_speed_1_5", F.when(col("speed") == 1.5, 1).otherwise(0)) 
    .withColumn("is_speed_gt_1_5", F.when(col("speed") > 1.5, 1).otherwise(0)) 
    .withColumn("is_speed_lt_1", F.when(col("speed") < 1.0, 1).otherwise(0))
)

# 5.2. Đếm tổng số lần sử dụng từng tốc độ
df_features = (
    df_features
    .withColumn("num_speed_1_user", F.sum("is_speed_1").over(window_user_course)) 
    .withColumn("num_speed_1_25_user", F.sum("is_speed_1_25").over(window_user_course)) 
    .withColumn("num_speed_1_5_user", F.sum("is_speed_1_5").over(window_user_course))
    .withColumn("num_speed_gt_1_5_user", F.sum("is_speed_gt_1_5").over(window_user_course)) 
    .withColumn("num_speed_lt_1_user", F.sum("is_speed_lt_1").over(window_user_course))
)

# 5.3. Tính tỷ lệ sử dụng từng tốc độ (trên tổng số sessions)
# (Lưu ý: total_sessions_user đã được tính ở bước 3)
# CHỈNH SỬA: Thêm F.when() để tránh lỗi chia cho 0 (mặc dù F.count("*") >= 1, nhưng đây là cách làm an toàn)
df_features =(
    df_features
    .withColumn("ratio_speed_1_user", 
        F.when(col("total_sessions_user") > 0, col("num_speed_1_user") / col("total_sessions_user"))
         .otherwise(0)
    ) 
    .withColumn("ratio_speed_1_25_user", 
        F.when(col("total_sessions_user") > 0, col("num_speed_1_25_user") / col("total_sessions_user"))
         .otherwise(0)
    ) 
    .withColumn("ratio_speed_1_5_user", 
        F.when(col("total_sessions_user") > 0, col("num_speed_1_5_user") / col("total_sessions_user"))
         .otherwise(0)
    ) 
    .withColumn("ratio_speed_gt_1_5_user", 
        F.when(col("total_sessions_user") > 0, col("num_speed_gt_1_5_user") / col("total_sessions_user"))
         .otherwise(0)
    ) 
    .withColumn("ratio_speed_lt_1_user", 
        F.when(col("total_sessions_user") > 0, col("num_speed_lt_1_user") / col("total_sessions_user"))
         .otherwise(0)
    )
)

print("Added speed features:")

Added speed features:


# 6. Tạo đặc trưng thống kê theo Phase 

In [43]:
# 6.1. Tính TỔNG THỜI GIAN XEM (watch_duration_real) cho từng phase
df_features = df_features.withColumn("total_watch_phase_1", 
    F.sum(F.when(col("phase_1") == 1, col("watch_duration_real")).otherwise(0)).over(window_user_course)
)
df_features = df_features.withColumn("total_watch_phase_2", 
    F.sum(F.when(col("phase_2") == 1, col("watch_duration_real")).otherwise(0)).over(window_user_course)
)
df_features = df_features.withColumn("total_watch_phase_3", 
    F.sum(F.when(col("phase_3") == 1, col("watch_duration_real")).otherwise(0)).over(window_user_course)
)
df_features = df_features.withColumn("total_watch_phase_4", 
    F.sum(F.when(col("phase_4") == 1, col("watch_duration_real")).otherwise(0)).over(window_user_course)
)

# 6.2. Tính TỶ LỆ THỜI GIAN XEM cho từng phase (so với tổng thời gian)
# (Lưu ý: total_watch_duration_real_user đã được tính ở bước 3)
df_features = df_features.withColumn("ratio_watch_phase_1", 
    F.when(col("total_watch_duration_real_user") == 0, 0)
     .otherwise(col("total_watch_phase_1") / col("total_watch_duration_real_user"))
)
df_features = df_features.withColumn("ratio_watch_phase_2", 
    F.when(col("total_watch_duration_real_user") == 0, 0)
     .otherwise(col("total_watch_phase_2") / col("total_watch_duration_real_user"))
)
df_features = df_features.withColumn("ratio_watch_phase_3", 
    F.when(col("total_watch_duration_real_user") == 0, 0)
     .otherwise(col("total_watch_phase_3") / col("total_watch_duration_real_user"))
)
df_features = df_features.withColumn("ratio_watch_phase_4", 
    F.when(col("total_watch_duration_real_user") == 0, 0)
     .otherwise(col("total_watch_phase_4") / col("total_watch_duration_real_user"))
)


# 6.3. Tính TỔNG SỐ SESSIONS (lần xem) cho từng phase
df_features = df_features.withColumn("total_sessions_phase_1", 
    F.sum(F.when(col("phase_1") == 1, 1).otherwise(0)).over(window_user_course)
)
df_features = df_features.withColumn("total_sessions_phase_2", 
    F.sum(F.when(col("phase_2") == 1, 1).otherwise(0)).over(window_user_course)
)
df_features = df_features.withColumn("total_sessions_phase_3", 
    F.sum(F.when(col("phase_3") == 1, 1).otherwise(0)).over(window_user_course)
)
df_features = df_features.withColumn("total_sessions_phase_4", 
    F.sum(F.when(col("phase_4") == 1, 1).otherwise(0)).over(window_user_course)
)

# 6.4. Tính TỶ LỆ SESSIONS cho từng phase (so với tổng số sessions)
# (Lưu ý: total_sessions_user đã được tính ở bước 3)
# CHỈNH SỬA: Thêm F.when() để tránh lỗi chia cho 0
df_features = df_features.withColumn("ratio_sessions_phase_1", 
    F.when(col("total_sessions_user") > 0, col("total_sessions_phase_1") / col("total_sessions_user"))
     .otherwise(0)
)
df_features = df_features.withColumn("ratio_sessions_phase_2", 
    F.when(col("total_sessions_user") > 0, col("total_sessions_phase_2") / col("total_sessions_user"))
     .otherwise(0)
)
df_features = df_features.withColumn("ratio_sessions_phase_3", 
    F.when(col("total_sessions_user") > 0, col("total_sessions_phase_3") / col("total_sessions_user"))
     .otherwise(0)
)
df_features = df_features.withColumn("ratio_sessions_phase_4", 
    F.when(col("total_sessions_user") > 0, col("total_sessions_phase_4") / col("total_sessions_user"))
     .otherwise(0)
)

print("Added features")

Added features


25/10/25 03:30:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------+---------+-----------+----------+--------+-------------+-------------------+----------+-----------+---------+-----+-------------------+--------------------+--------------+----------------+-------+-------+-------+-------+-----------------+-------------------+------------------------------+-------------------+--------------+----------------------------+--------------+-----+--------+---------------+----------------+-----------------+-----------------------+------------------------+----------+-------------+------------+---------------+-------------+----------------+-------------------+------------------+---------------------+-------------------+------------------+---------------------+--------------------+-----------------------+---------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------------------+----------------------+-------------------

In [ ]:
final_feature_columns = [
    # --- Keys & Info (Thông tin tĩnh của user-course) ---
    "user_id", "course_id", 
    "certificate", "start_date", "end_date", "duration_days", "enroll_time", "remain_day",
    
    # --- General Stats (Thống kê chung - Bước 3) ---
    "total_watch_duration_real_user", 
    "total_sessions_user", 
    "avg_speed_user", 
    "avg_watch_duration_real_user",
    
    # --- Behavior Stats (Thống kê hành vi - Bước 4) ---
    "total_pauses_user", 
    "total_seek_forward_user", 
    "total_seek_backward_user",
    
    # --- Speed Ratios (Tỷ lệ tốc độ - Bước 5) ---
    "ratio_speed_1_user", 
    "ratio_speed_1_25_user", 
    "ratio_speed_1_5_user", 
    "ratio_speed_gt_1_5_user", 
    "ratio_speed_lt_1_user",
    
    # --- Phase Watch Time Ratios (Tỷ lệ thời gian xem theo Phase - Bước 6) ---
    "ratio_watch_phase_1", 
    "ratio_watch_phase_2", 
    "ratio_watch_phase_3", 
    "ratio_watch_phase_4",
    
    # --- Phase Session Ratios (Tỷ lệ số lần xem theo Phase - Bước 6) ---
    "ratio_sessions_phase_1", 
    "ratio_sessions_phase_2", 
    "ratio_sessions_phase_3", 
    "ratio_sessions_phase_4"
]

In [ ]:
fina

In [ ]:
uv_null_ratio = df_user_video_final.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_video_final.columns
])
uv_null_ratio.show()

# Thử Nghiệm 

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType, TimestampType

# === KHỐI 1: MÃ DO NGƯỜI DÙNG CUNG CẤP (PHẦN SETUP) ===
# Giả định các DataFrame 'video_info_cleaned', 'df_video_course_map_clean', 
# 'df_user_video_final', 'df_user_course_clean' đã tồn tại.

print("Bắt đầu Khối 1: Setup dữ liệu (từ code đã cung cấp)")

# 1. Chuẩn bị Dữ liệu Video
df_video_durations = video_info_cleaned.select(
    F.col("video_id"),
    F.col("video_duration")
).distinct()

# 2. Chuẩn bị Dữ liệu Khóa học
df_course_video_details = df_video_course_map_clean.join(
    df_video_durations,
    "video_id",
    "left"
)
df_course_stats = df_course_video_details.groupBy("course_id").agg(
    F.collect_list("video_id").alias("course_video_list"),
    F.sum("video_duration").alias("total_video_duration"),
    F.count("video_id").alias("num_videos_in_course")
)

# 3. Chuẩn bị Dữ liệu Xem của Người dùng-Khóa học
df_user_watch_by_course = df_user_video_final.join(
    df_video_course_map_clean,
    "video_id",
    "inner"
)

# 4. Tinh chỉnh Dữ liệu Sự kiện (Event)
df_user_watch_events = df_user_watch_by_course.withColumnRenamed("video_duration", "log_video_duration")
df_user_watch_events = df_user_watch_events.join(
    df_video_durations,
    "video_id",
    "left"
)
df_user_watch_events = df_user_watch_events.withColumn(
    "video_completion_rate_event",
    F.when(F.col("log_video_duration") > 0, F.col("actual_watch_time") / F.col("log_video_duration"))
     .otherwise(0)
).withColumn(
    "is_rewatch",
    F.when(F.col("rewind_time") > 0, 1).otherwise(0)
)
print("Hoàn thành Khối 1.")


# === KHỐI 2: TÍNH TOÁN 5 PHASE (CHỈ DỮ LIỆU HOẠT ĐỘNG) ===
print("Bắt đầu Khối 2: Tính toán 5 phase (chỉ dòng hoạt động).")

# 1. Thêm 'enroll_time' và tính 'day_since_enroll_event' cho TỪNG SỰ KIỆN
df_events_with_enroll = df_user_watch_events.join(
    df_user_course_clean.select("user_id", "course_id", "enroll_time"),
    ["user_id", "course_id"],
    "left"
)

df_events_with_enroll = df_events_with_enroll.withColumn(
    "day_since_enroll_event",
    F.when(
        F.col("first_local_start_time").isNotNull() & F.col("enroll_time").isNotNull(),
        F.datediff(
            F.col("first_local_start_time").cast("date"),
            F.col("enroll_time").cast("date")
        )
    ).otherwise(None)
)

# 2. Tính 'day_since_enroll' TỔNG QUÁT (Overall)
# Dùng để thêm vào file CSV cho biết ngày xem đầu tiên là ngày nào
df_user_first_watch_stats = df_events_with_enroll.groupBy("user_id", "course_id").agg(
    F.min("first_local_start_time").alias("first_local_start_time"),
    F.min("day_since_enroll_event").alias("day_since_enroll")
)

# 3. Xác định các Phase
phases = [
    (0, 14, "phase1"),
    (15, 28, "phase2"),
    (29, 42, "phase3"),
    (43, 56, "phase4"),
    (57, None, "phase5") # <<< THAY ĐỔI: THÊM PHASE 5 (TỪ NGÀY 57 TRỞ ĐI)
]

# Lấy các cột fillna (từ code của user)
numeric_fill_cols = {
    "total_video_duration": 0.0,
    "num_videos_in_course": 0,
    "avg_watched_duration": 0.0,
    "avg_speed": 0.0,
    "avg_completion_rate_per_video": 0.0
}

# Lấy các cột tỷ lệ (từ code của user)
rate_cols = [
    "completion_rate",
    "unique_videos_ratio",
    "rewatch_ratio_original",
    "rewatch_sessions_ratio",
    "avg_completion_rate_per_video",
    "video_activity_ratio"
]

# Lấy hằng số capping (từ code của user)
MAX_COMPLETION = 2.0
MAX_RATIO = 1.0

# Định nghĩa các cột cuối cùng (sửa đổi từ code user)
final_columns = [
    "user_id", "course_id", "certificate", "enroll_time",
    "start_date", "end_date", "duration_days", "remain_day",
    "first_local_start_time", # Ngày xem đầu tiên (overall)
    "day_since_enroll",       # Số ngày từ enroll đến ngày xem đầu tiên (overall)
    "user_video_list",        # List video đã xem TRONG PHASE NÀY
    "avg_watched_duration",   # Chỉ số của PHASE NÀY
    "avg_speed",              # Chỉ số của PHASE NÀY
    "num_videos_watched",     # Chỉ số của PHASE NÀY
    "num_sessions",           # Chỉ số của PHASE NÀY
    "total_watched_duration", # Chỉ số của PHASE NÀY
    "course_video_list",
    "total_video_duration",
    "num_common_videos",
    "estimated_watched_in_course",
    "completion_rate",
    "unique_videos_ratio",
    "rewatch_ratio_original",
    "rewatch_sessions_ratio",
    "avg_completion_rate_per_video",
    "video_activity_ratio"
]

print("Bắt đầu xử lý 5 phase...")

# 4. Vòng lặp tính toán và xuất CSV cho từng Phase
for start_day, end_day, phase_name in phases:
    print(f"--- Đang xử lý {phase_name} ---")

    # 4.1. Lọc sự kiện (event) cho phase này
    # <<< THAY ĐỔI: Thêm logic để xử lý end_day = None (cho phase 5) >>>
    print(f"Lọc sự kiện từ ngày {start_day}" + (f" đến {end_day}" if end_day is not None else " trở đi"))
        
    if end_day is not None:
        # Logic cho Phase 1-4
        df_events_phase = df_events_with_enroll.filter(
            (F.col("day_since_enroll_event") >= start_day) &
            (F.col("day_since_enroll_event") <= end_day)
        )
    else:
        # Logic cho Phase 5 (end_day is None)
        df_events_phase = df_events_with_enroll.filter(
            (F.col("day_since_enroll_event") >= start_day)
        )
    # <<< KẾT THÚC THAY ĐỔI >>>

    # 4.2. Tổng hợp hành vi (Chỉ những user hoạt động)
    df_watch_stats_phase = df_events_phase.groupBy("user_id", "course_id").agg(
        F.collect_list("video_id").alias("user_video_list_with_duplicates"),
        F.sum("actual_watch_time").alias("total_watched_duration"),
        F.avg("actual_watch_time").alias("avg_watched_duration"),
        F.avg("average_speed").alias("avg_speed"),
        F.count("video_id").alias("num_sessions"),
        F.countDistinct("video_id").alias("num_videos_watched"),
        F.sum("is_rewatch").alias("num_rewatches"),
        F.avg("video_completion_rate_event").alias("avg_completion_rate_per_video")
    ).withColumn(
        "user_video_list", F.array_distinct("user_video_list_with_duplicates")
    ).drop("user_video_list_with_duplicates")

    df_watch_stats_phase.cache()
    
    active_rows_count = df_watch_stats_phase.count()
    if active_rows_count == 0:
        print(f"Không có hoạt động nào trong {phase_name}. Bỏ qua...")
        df_watch_stats_phase.unpersist()
        continue
    
    print(f"Tìm thấy {active_rows_count} user-course hoạt động trong {phase_name}.")

    # 4.3. Gộp thông tin tĩnh (static) và thông tin khóa học
    df_final_agg_phase = df_watch_stats_phase.join(
        df_user_course_clean, 
        ["user_id", "course_id"],
        "left"
    ).join(
        df_course_stats, 
        "course_id",
        "left"
    ).join(
        df_user_first_watch_stats, 
        ["user_id", "course_id"],
        "left"
    )

    # 4.4. Xử lý giá trị NULL
    df_final_agg_phase = df_final_agg_phase.fillna(numeric_fill_cols)
    df_final_agg_phase = df_final_agg_phase.withColumn(
        "course_video_list",
        F.when(F.col("course_video_list").isNull(), F.array().cast(ArrayType(StringType())))
        .otherwise(F.col("course_video_list"))
    )

    # 4.5. Tính toán các chỉ số cuối cùng
    df_final_output_raw_phase = df_final_agg_phase.withColumn(
        "num_common_videos",
        F.col("num_videos_watched")
    ).withColumn(
        "estimated_watched_in_course",
        F.col("total_watched_duration")
    ).withColumn(
        "completion_rate_raw",
        F.when(F.col("total_video_duration") > 0, F.col("total_watched_duration") / F.col("total_video_duration"))
        .otherwise(0.0)
    ).withColumn(
        "unique_videos_ratio_raw",
        F.when(F.col("num_videos_in_course") > 0, F.col("num_videos_watched") / F.col("num_videos_in_course"))
        .otherwise(0.0)
    ).withColumn(
        "rewatch_ratio_original",
        F.when(F.col("num_sessions") > 0, F.col("num_rewatches") / F.col("num_sessions"))
        .otherwise(0.0)
    ).withColumn(
        "rewatch_sessions_ratio",
        F.when(
            (F.col("num_sessions") > 0) & (F.col("num_sessions") > F.col("num_videos_watched")),
            (F.col("num_sessions") - F.col("num_videos_watched")) / F.col("num_sessions")
        ).otherwise(0.0)
    ).withColumn(
        "video_activity_ratio",
        F.when(F.col("num_videos_in_course") > 0, F.col("num_sessions") / F.col("num_videos_in_course"))
        .otherwise(0.0)
    )
    
    # 4.6. Chuẩn hóa (Capping)
    df_final_output_phase = df_final_output_raw_phase.withColumn(
        "completion_rate",
        F.when(F.col("completion_rate_raw") > MAX_COMPLETION, MAX_COMPLETION)
        .otherwise(F.col("completion_rate_raw"))
    ).withColumn(
        "unique_videos_ratio",
        F.when(F.col("unique_videos_ratio_raw") > MAX_RATIO, MAX_RATIO)
        .otherwise(F.col("unique_videos_ratio_raw"))
    )

    # 4.7. Sắp xếp lại các cột
    df_final_output_phase = df_final_output_phase.select(final_columns)

    # 4.8. Định dạng %
    for c in rate_cols:
        df_final_output_phase = df_final_output_phase.withColumn(
            c,
            F.concat(F.round(F.col(c) * 100, 2).cast("string"), F.lit("%"))
        )
    
    # 4.9. Xử lý cột ArrayType (chuyển sang string)
    df_for_csv = df_final_output_phase.withColumn(
        "user_video_list", 
        F.concat_ws("|", F.col("user_video_list"))
    ).withColumn(
        "course_video_list", 
        F.concat_ws("|", F.col("course_video_list"))
    )

    # 4.10. Xuất ra 1 file CSV cho phase này
    output_path = f"/kaggle/working/df_final_output_{phase_name}_active_only"
    df_for_csv.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path)
    
    print(f"Đã lưu file CSV cho {phase_name} tại: {output_path}")
    
    df_watch_stats_phase.unpersist()

print("--- Hoàn thành xử lý 5 phase (chỉ user hoạt động)! ---")

In [ ]:
# Lọc để chỉ hiển thị những hàng có thời gian xem lớn hơn 0
df_active_users = df_final_output.filter(F.col("total_watched_duration") > 0)

df_active_users.show()

In [ ]:
total = df_final_output.count()

null_ratio = df_final_output.select([
    (F.count(F.when(F.col(c).isNull(), 1)) / total * 100).alias(c) for c in df_final_output.columns
])
null_ratio.show()

In [ ]:
# from pyspark.sql import functions as F

# # Tạo một DataFrame mới để chuẩn bị lưu CSV
# # Chuyển đổi các cột mảng thành chuỗi, phân tách bằng dấu '|'
# df_for_csv = df_final_output.withColumn(
#     "user_video_list", 
#     F.concat_ws("|", F.col("user_video_list"))
# ).withColumn(
#     "course_video_list", 
#     F.concat_ws("|", F.col("course_video_list"))
# )

# # Bây giờ tất cả các cột đều là kiểu nguyên thủy, có thể lưu ra CSV
# df_for_csv.coalesce(1).write.mode("overwrite").option("header", "true").csv("/kaggle/working/df_final_output")

# print("Đã lưu file CSV thành công!")

In [ ]:
# import shutil

# input_dir = "/kaggle/working/df_final_output"
# output_zip = "/kaggle/working/df_final_output"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_final_output.zip")


In [ ]:
# !rm -rf /kaggle/working/*
